# prepare enviro of RDD

In [1]:
from pyspark.sql import *

In [2]:
SparkContext

pyspark.context.SparkContext

In [3]:
SparkSession


pyspark.sql.session.SparkSession

In [4]:
spark=SparkSession.builder.getOrCreate()


In [5]:
spark

In [6]:
sc=spark.sparkContext

In [7]:
sc

<SparkContext master=local[*] appName=PySparkShell>

# extract file form any file

In [4]:
data=sc.textFile("bigdatafile.json")

In [5]:
data.getNumPartitions()

4

In [6]:
data.take(10) # as list 

['{"InvoiceNo":536365,"InvoiceDate":1291161600000,"CustomerID":17850,"branch_name":"United Kingdom","branch_id":36,"salesperson_id":57,"product_id":3367,"product_name":"85123A","Description":"WHITE HANGING HEART T-LIGHT HOLDER","quantity_sold":6,"UnitPrice":2.55,"totalsales":15.3}',
 '{"InvoiceNo":536365,"InvoiceDate":1291161600000,"CustomerID":17850,"branch_name":"United Kingdom","branch_id":36,"salesperson_id":74,"product_id":2432,"product_name":71053,"Description":"WHITE METAL LANTERN","quantity_sold":6,"UnitPrice":3.39,"totalsales":20.34}',
 '{"InvoiceNo":536365,"InvoiceDate":1291161600000,"CustomerID":17850,"branch_name":"United Kingdom","branch_id":36,"salesperson_id":71,"product_id":3129,"product_name":"84406B","Description":"CREAM CUPID HEARTS COAT HANGER","quantity_sold":8,"UnitPrice":2.75,"totalsales":22.0}',
 '{"InvoiceNo":536365,"InvoiceDate":1291161600000,"CustomerID":17850,"branch_name":"United Kingdom","branch_id":36,"salesperson_id":92,"product_id":3099,"product_name":"

In [7]:
# Parse JSON and Create a Dictionary
json_rdd = data.map(lambda x: json.loads(x)) # convert each json string to python dictionary  


In [8]:
json_rdd.take(10)

[{'InvoiceNo': 536365,
  'InvoiceDate': 1291161600000,
  'CustomerID': 17850,
  'branch_name': 'United Kingdom',
  'branch_id': 36,
  'salesperson_id': 57,
  'product_id': 3367,
  'product_name': '85123A',
  'Description': 'WHITE HANGING HEART T-LIGHT HOLDER',
  'quantity_sold': 6,
  'UnitPrice': 2.55,
  'totalsales': 15.3},
 {'InvoiceNo': 536365,
  'InvoiceDate': 1291161600000,
  'CustomerID': 17850,
  'branch_name': 'United Kingdom',
  'branch_id': 36,
  'salesperson_id': 74,
  'product_id': 2432,
  'product_name': 71053,
  'Description': 'WHITE METAL LANTERN',
  'quantity_sold': 6,
  'UnitPrice': 3.39,
  'totalsales': 20.34},
 {'InvoiceNo': 536365,
  'InvoiceDate': 1291161600000,
  'CustomerID': 17850,
  'branch_name': 'United Kingdom',
  'branch_id': 36,
  'salesperson_id': 71,
  'product_id': 3129,
  'product_name': '84406B',
  'Description': 'CREAM CUPID HEARTS COAT HANGER',
  'quantity_sold': 8,
  'UnitPrice': 2.75,
  'totalsales': 22.0},
 {'InvoiceNo': 536365,
  'InvoiceDate': 

## Analysis on RDD 

# 1 Count Total Records

In [9]:
total_records=json_rdd.count()

In [10]:
print(f"Total Records: {total_records}")

Total Records: 406829


 # 2 Get Unique Customer IDs

In [11]:
unique_customers = json_rdd.map(lambda x: x['CustomerID']).distinct().count()

In [12]:
print(f"Unique Customers: {unique_customers}")

Unique Customers: 4372


# 3 Calculate Total Sales

In [13]:
total_sales = json_rdd.map(lambda x: x['totalsales']).reduce(lambda x, y: x + y)

# reduce is used to aggregate all elements of an RDD into a single value.

In [16]:
print(f'total sales of {total_records} record for {unique_customers:,} customers is {total_sales:,.2f} $')

total sales of 406829 record for 4,372 customers is 8,300,065.81 $


# 4 Find Sales by Product(TOP20)

In [63]:
sales_prod_desc = json_rdd.map(lambda x: (x['product_name'], (x['totalsales'], x['Description']))) \
    .reduceByKey(lambda a, b: (a[0] + b[0], a[1]))


#(x[0]is product ,(x[1][0]is total sales,x[1][1]descri)) 
sorted_sales_prod = sales_prod_desc.map(lambda x: (x[0], (x[1][0], x[1][1])))\
.sortBy(lambda x: x[1][0], ascending=False)  # Sort by total_sales (desc from large to small) #x[1]because he want total sales 

sorted_sales.take(20) # as list but each elemnt as tuple key_paires (,())

#reduceByKey is used specifically with key-value pair RDDs

[(22423, (132870.39999999994, 'REGENCY CAKESTAND 3 TIER')),
 ('85123A', (93979.20000000007, 'WHITE HANGING HEART T-LIGHT HOLDER')),
 ('85099B', (83236.75999999975, 'JUMBO BAG RED RETROSPOT')),
 (47566, (67687.52999999997, 'PARTY BUNTING')),
 ('POST', (66710.24, 'POSTAGE')),
 (84879, (56499.2200000003, 'ASSORTED COLOUR BIRD ORNAMENT')),
 (23084, (51137.79999999986, 'RABBIT NIGHT LIGHT')),
 (22502, (46980.95, 'PICNIC BASKET WICKER SMALL')),
 (79321, (45936.80999999999, 'CHILLI LIGHTS')),
 (22086, (41500.48000000011, "PAPER CHAIN KIT 50'S CHRISTMAS ")),
 (21137, (39009.38, 'BLACK RECORD COVER FRAME')),
 (22197, (36859.87999999999, 'SMALL POPCORN HOLDER')),
 (23203, (36607.97999999985, 'JUMBO BAG DOILEY PATTERNS')),
 (22386, (36473.0099999999, 'JUMBO BAG PINK POLKADOT')),
 (23298, (35691.140000000014, 'SPOTTY BUNTING')),
 (23284, (34312.59999999997, 'DOORMAT KEEP CALM AND COME IN')),
 (82484, (34307.05999999996, 'WOOD BLACK BOARD ANT WHITE FINISH')),
 (22720, (32607.800000000138, 'SET OF 3

In [68]:
countprod=sorted_sales_prod.count()
print(countprod)

3684


In [69]:

# Print the header
print(f'the top20 product in totalsales  from {countprod}\n')

# Iterate through the collected list
for product,(total_sales, desc) in sorted_sales_prod.take(20): # as list 
    print(f"The name of product '{product}' has the description '{desc}' and total sales is: {total_sales:,.02f} $")


the top20 product in totalsales  from 3684

The name of product '22423' has the description 'REGENCY CAKESTAND 3 TIER' and total sales is: 132,870.40 $
The name of product '85123A' has the description 'WHITE HANGING HEART T-LIGHT HOLDER' and total sales is: 93,979.20 $
The name of product '85099B' has the description 'JUMBO BAG RED RETROSPOT' and total sales is: 83,236.76 $
The name of product '47566' has the description 'PARTY BUNTING' and total sales is: 67,687.53 $
The name of product 'POST' has the description 'POSTAGE' and total sales is: 66,710.24 $
The name of product '84879' has the description 'ASSORTED COLOUR BIRD ORNAMENT' and total sales is: 56,499.22 $
The name of product '23084' has the description 'RABBIT NIGHT LIGHT' and total sales is: 51,137.80 $
The name of product '22502' has the description 'PICNIC BASKET WICKER SMALL' and total sales is: 46,980.95 $
The name of product '79321' has the description 'CHILLI LIGHTS' and total sales is: 45,936.81 $
The name of product 

#  5 find sales by branch(TOP10)  

In [70]:

sales_branch = json_rdd.map(lambda x: (x['branch_name'],(x['branch_id'], x['totalsales'])))\
.reduceByKey(lambda x, y: (x[0],(x[1]+y[1])) )
sales_branch.collect()

sorted_sales_branch=sales_branch.map(lambda x:(x[0],(x[1][0],x[1][1]))).sortBy(lambda x:x[1][1],ascending=False)

sorted_sales_branch.take(20)


[('United Kingdom', (36, 6767873.393999164)),
 ('Netherlands', (24, 284661.53999999975)),
 ('EIRE', (11, 250285.21999999945)),
 ('Germany', (15, 221698.21000000072)),
 ('France', (14, 196712.84)),
 ('Australia', (1, 137077.26999999996)),
 ('Switzerland', (33, 55739.40000000005)),
 ('Spain', (31, 54774.58000000008)),
 ('Belgium', (4, 40910.95999999999)),
 ('Sweden', (32, 36595.909999999996)),
 ('Japan', (20, 35340.619999999995)),
 ('Norway', (25, 35163.46000000001)),
 ('Portugal', (27, 29059.809999999983)),
 ('Finland', (13, 22326.740000000005)),
 ('Channel Islands', (7, 20086.289999999997)),
 ('Denmark', (10, 18768.139999999992)),
 ('Italy', (19, 16890.509999999995)),
 ('Cyprus', (8, 12946.289999999992)),
 ('Austria', (2, 10154.320000000003)),
 ('Singapore', (30, 9120.390000000001))]

In [71]:
countbranch=sales_branch.count()
countbranch

37

In [72]:

# Print the header
print(f'the top 10 branch in totalsales  from {countbranch}\n')

# Iterate through the collected list
for branch,(id_b,sales) in sorted_sales_branch.take(20): # as list 
    print(f"The name of branch '{branch}' his id is  '{id_b}' and his total sales  is: {sales:,.02f} $")

the top 10 branch in totalsales  from 37

The name of branch 'United Kingdom' his id is  '36' and his total sales  is: 6,767,873.39 $
The name of branch 'Netherlands' his id is  '24' and his total sales  is: 284,661.54 $
The name of branch 'EIRE' his id is  '11' and his total sales  is: 250,285.22 $
The name of branch 'Germany' his id is  '15' and his total sales  is: 221,698.21 $
The name of branch 'France' his id is  '14' and his total sales  is: 196,712.84 $
The name of branch 'Australia' his id is  '1' and his total sales  is: 137,077.27 $
The name of branch 'Switzerland' his id is  '33' and his total sales  is: 55,739.40 $
The name of branch 'Spain' his id is  '31' and his total sales  is: 54,774.58 $
The name of branch 'Belgium' his id is  '4' and his total sales  is: 40,910.96 $
The name of branch 'Sweden' his id is  '32' and his total sales  is: 36,595.91 $
The name of branch 'Japan' his id is  '20' and his total sales  is: 35,340.62 $
The name of branch 'Norway' his id is  '25

# 6 total sales according name of branch

In [74]:
# Filter by Branch
uk_sales = json_rdd.filter(lambda x: x['branch_name'] == "United Kingdom")
uk_sales.take(20)



[{'InvoiceNo': 536365,
  'InvoiceDate': 1291161600000,
  'CustomerID': 17850,
  'branch_name': 'United Kingdom',
  'branch_id': 36,
  'salesperson_id': 57,
  'product_id': 3367,
  'product_name': '85123A',
  'Description': 'WHITE HANGING HEART T-LIGHT HOLDER',
  'quantity_sold': 6,
  'UnitPrice': 2.55,
  'totalsales': 15.3},
 {'InvoiceNo': 536365,
  'InvoiceDate': 1291161600000,
  'CustomerID': 17850,
  'branch_name': 'United Kingdom',
  'branch_id': 36,
  'salesperson_id': 74,
  'product_id': 2432,
  'product_name': 71053,
  'Description': 'WHITE METAL LANTERN',
  'quantity_sold': 6,
  'UnitPrice': 3.39,
  'totalsales': 20.34},
 {'InvoiceNo': 536365,
  'InvoiceDate': 1291161600000,
  'CustomerID': 17850,
  'branch_name': 'United Kingdom',
  'branch_id': 36,
  'salesperson_id': 71,
  'product_id': 3129,
  'product_name': '84406B',
  'Description': 'CREAM CUPID HEARTS COAT HANGER',
  'quantity_sold': 8,
  'UnitPrice': 2.75,
  'totalsales': 22.0},
 {'InvoiceNo': 536365,
  'InvoiceDate': 

In [77]:
uk_sales_count=uk_sales.count()
uk_sales_count

361878

In [79]:
uk_total_sales = uk_sales.map(lambda x: x['totalsales']).reduce(lambda x, y: x + y)
print(f"Total Sales in UK at {uk_sales_count} records is: {uk_total_sales:,.02f} $")


Total Sales in UK at 361878 records is: 6,767,873.39 $


#  7 Calculate Average Sales

In [82]:
# Calculate Average Sales
avg_sales = json_rdd.map(lambda x: x['totalsales']).mean()
print(f"Average Sales at {total_records} is : {avg_sales:,.03f} $")


Average Sales at 406829 is : 20.402 $
